In [1]:
from module.engine.tensor import Tensor
from module.model.layers import Layer , Sequential
from module.model.loss import Loss
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

## Solving a regression problem with the sklearn diabetes dataset

In [2]:
#load data
X , y = datasets.load_diabetes(return_X_y=True, as_frame=False, scaled=True)

#split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [3]:
X[0]

array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
       -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613])

In [6]:
model = Sequential([Layer(size=(10,20), activation='relu'),
                    Layer(size=(20,20) ,activation='relu'),
                    Layer(size=(20,5) ,activation='relu'),
                    Layer(size=(5,1))])
model.summary()

Layer(type)         Shape              Parameters
---------------------------------------------------
ReLULayer       Shape: (10, 20))      Params: 220
---------------------------------------------------
ReLULayer       Shape: (20, 20))      Params: 420
---------------------------------------------------
ReLULayer       Shape: (20, 5))      Params: 105
---------------------------------------------------
LinearLayer       Shape: (5, 1))      Params: 6
---------------------------------------------------
Trainable Parameters                          751


In [40]:
len(X_train)/70

4.414285714285715

In [4]:
lossi = [] 
niter = 1000
bactch_size = 30
lr = 0.01
def train_model(split):
    X, Y = {'train': (X_train, y_train), 'test': (X_test, y_test)}[split]
    for t in range(niter):
        size = np.random.randint(0, len(X), bactch_size)
        ypred = [model(X[x]) for x in size]
        loss = sum([Loss.mean_squared_error(y[size], ypred)])
        lossi.append(loss.data)
        if t % 50 == 0:
            print(f'Epoch {t} loss: {loss.data}')
            model.zero_grad()
            loss.backward()
            for p in model.parameters():
                p.data += - lr * p.grad 

In [7]:
train_model('train')

Epoch 0 loss: 19352.77585502553
Epoch 50 loss: 25758.569131716293
Epoch 100 loss: 15583.888465194352


In [3]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] 
n = Sequential([Layer(size=(3,4), activation='tanh'),
                    Layer(size=(4,4) ,activation='tanh'),
                    Layer(size=(4,1), activation='tanh')])
n.summary()
for k in range(20): 
    p = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, p))
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.1 * p.grad
    print(k, loss.data)